In [ ]:
# from models.sos import ScalarOnScalarModel
# from optimizers.nbdo import NBDO
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# import numpy as np

In [ ]:
# # SoS
# model = ScalarOnScalarModel(Kx=3, criterion="A", order=2)
# nbdo = NBDO(model=model, latent_dim=10, seed=42)
# nbdo.compute_train_set(num_designs=1_000, runs=25)
# nbdo.fit(epochs=1_000, patience=100, batch_size=256)
# crit, design = nbdo.optimize(n_calls=30)
# np.round(crit,2)

In [ ]:
# # SoF
# xB, bB = BSplineBasis(0,4), BSplineBasis(0,2)
# model = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True)
# nbdo = NBDO(model=model, latent_dim=2, seed=42, verbose=False)
# nbdo.compute_train_set(num_designs=1_000, runs=12)
# nbdo.fit(epochs=10, patience=5, batch_size=256)
# crit, design = nbdo.optimize(n_calls=5)
# crit

In [ ]:
# from diagnostics import (
#     info_matrix, eigen_spectrum, condition_number,
#     leverage_diag, prediction_variance,
# )

# M = info_matrix(model, design)
# eigvals, _ = eigen_spectrum(M)
# kappa = condition_number(M)
# print(f"p = {M.shape[0]}, λ_min = {eigvals[-1]:.3e}, λ_max = {eigvals[0]:.3e}, κ2 = {kappa:.2e}")

# h = leverage_diag(model, design)
# print(f"leverage stats: min={h.min():.3f}, max={h.max():.3f}, mean={h.mean():.3f}")

In [ ]:
# p = 10, λ_min = 3.458e+00, λ_max = 5.728e+01, κ2 = 1.66e+01
# leverage stats: min=0.179, max=0.748, mean=0.400

In [ ]:
# # quick smoke test
# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# def B(K, deg):
#     return BSplineBasis(degree=deg, total_knots_num=max(2, K - deg + 1))

# bx, bb = B(5,2), B(6,2)
# model = FunctionOnFunctionModel([(bx, bb)], criterion="A", intercept=True)
# opt = NBDO(model=model, latent_dim=4, seed=0)
# opt.compute_train_set(num_designs=1_000, runs=12)
# opt.fit(epochs=1_000, patience=100, batch_size=256)       # should converge & print losses
# report, design = opt.optimize(n_calls=5, n_random_starts=3)
# print("ok ✅", report, design.shape)

In [ ]:
# import numpy as np
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from models.fof import FunctionOnFunctionModel

# def B(K, deg):
#     total_knots_num = max(2, K - deg + 1)
#     return BSplineBasis(degree=deg, total_knots_num=total_knots_num)

# # Two predictors, different dims
# bx1, bb1 = B(5,2), B(6,2)
# bx2, bb2 = B(4,3), B(3,3)
# pairs = [(bx1, bb1), (bx2, bb2)]

# runs = 14

# fof = FunctionOnFunctionModel(pairs, criterion="A", intercept=True)
# sof = ScalarOnFunctionModel(pairs, criterion="A", intercept=True)

# # Build Γ by concatenating per-predictor blocks
# rng = np.random.default_rng(0)
# Gamma = np.hstack([
#     rng.normal(size=(runs, fof.Kx_list[0])),
#     rng.normal(size=(runs, fof.Kx_list[1])),
# ])

# # Block-diag sanity: each slice maps correctly
# Phi = Gamma @ fof.J_np
# for xs, bs in zip(fof.x_slices, fof.b_slices):
#     Ji = fof.J_np[xs, :][:, bs]
#     assert np.allclose(Gamma[:, xs] @ Ji, Phi[:, bs], atol=1e-12)

# # Shapes and parity
# Zf, Zs = fof.model_matrix(Gamma), sof.model_matrix(Gamma)
# Mf, Ms = fof.information_matrix(Zf), sof.information_matrix(Zs)
# print("Z shapes:", Zf.shape, Zs.shape)
# print("M shapes:", Mf.shape, Ms.shape)

# vf, vs = fof.objective_num(Gamma), sof.objective_num(Gamma)
# print("A-opt FoF vs SoF:", vf, vs)
# assert np.allclose(vf, vs, rtol=1e-10, atol=1e-10)

# print("Multi-predictor behavior matches SoF ✅")

In [ ]:
# # --- Scalar-on-Function parity baseline ---

# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel

# # reproducibility
# rng = np.random.default_rng(12345)
# tf.keras.utils.set_random_seed(12345)

# def B(K: int, degree: int) -> BSplineBasis:
#     """
#     Your BSplineBasis signature: (degree, total_knots_num)
#     Relation: K = degree + total_knots_num - 1  -> total_knots_num = K - degree + 1
#     """
#     total_knots_num = max(2, K - degree + 1)
#     return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# # --- common design setup (multi-predictor) ---
# deg1, deg2 = 2, 2
# Kx1, Kb1 = 5, 6    # predictor 1: design basis size, beta(s,·) s-basis size
# Kx2, Kb2 = 4, 3    # predictor 2: design basis size, beta(s,·) s-basis size

# bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
# bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
# pairs = [(bx1, bb1), (bx2, bb2)]

# intercept = True
# Kb_total = Kb1 + Kb2
# p = Kb_total + (1 if intercept else 0)

# runs = 14  # ensure runs > p for a well-conditioned M

# # --- build model(s) ---
# sof_A = ScalarOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64)
# sof_D = ScalarOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64)

# # --- design coefficients Γ: (runs, Kx_total) ---
# Gamma = rng.normal(size=(runs, sof_A.Kx))

# # --- shapes & quick info ---
# print("=== Scalar-on-Function (baseline) ===")
# print(f"Kx_list={sof_A.Kx_list}, sum={sof_A.Kx} | Kb_list={sof_A.Kb_list}, sum={sof_A.Kb}")
# print(f"p={sof_A.p} (should equal {p}), runs={runs}")
# print("J shape:", sof_A.J_np.shape)

# Z = sof_A.model_matrix(Gamma)
# M = sof_A.information_matrix(Z)
# print("Z shape:", Z.shape, "| M shape:", M.shape)

# # --- criteria values ---
# A_val = sof_A.objective_num(Gamma)
# D_val = sof_D.objective_num(Gamma)

# print("\nResults to copy:")
# print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
# print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


In [ ]:
# # --- Function-on-Function parity check (should match SoF numbers) ---

# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel

# # reproducibility (same seeds as in Cell 1)
# rng = np.random.default_rng(12345)
# tf.keras.utils.set_random_seed(12345)

# def B(K: int, degree: int) -> BSplineBasis:
#     total_knots_num = max(2, K - degree + 1)
#     return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# # --- same common design setup (multi-predictor) ---
# deg1, deg2 = 2, 2
# Kx1, Kb1 = 5, 6
# Kx2, Kb2 = 4, 3

# bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
# bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
# pairs = [(bx1, bb1), (bx2, bb2)]

# intercept = True
# Kb_total = Kb1 + Kb2
# p = Kb_total + (1 if intercept else 0)

# runs = 14  # same as Cell 1

# # --- build model(s) ---
# fof_A = FunctionOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64, response_basis=None)
# fof_D = FunctionOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64, response_basis=None)

# # --- design coefficients Γ: (runs, Kx_total) -- same RNG/state as Cell 1 ---
# Gamma = rng.normal(size=(runs, fof_A.Kx))

# # --- shapes & quick info ---
# print("=== Function-on-Function (to match SoF) ===")
# print(f"Kx_list={fof_A.Kx_list}, sum={fof_A.Kx} | Kb_list={fof_A.Kb_list}, sum={fof_A.Kb}")
# print(f"p={fof_A.p} (should equal {p}), runs={runs}")
# print("J shape:", fof_A.J_np.shape)

# Z = fof_A.model_matrix(Gamma)
# M = fof_A.information_matrix(Z)
# print("Z shape:", Z.shape, "| M shape:", M.shape)

# # --- criteria values (should match those you copied from Cell 1) ---
# A_val = fof_A.objective_num(Gamma)
# D_val = fof_D.objective_num(Gamma)

# print("\nCompare to your SoF notes:")
# print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
# print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


In [ ]:
# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from inner_products.penalty import PenaltyBuilder

# # --- Build a simple 1-factor setup ---
# xB = BSplineBasis(degree=3, total_knots_num=12)  # input basis (Kx)
# bB = BSplineBasis(degree=3, total_knots_num=12)  # parameter basis (Kb)
# basis_pairs = [(xB, bB), (xB, bB)]

# # --- PenaltyBuilder sanity ---
# pb = PenaltyBuilder([bB], quad_points=256, diff_h=1e-4)
# R0 = pb.build()
# print("R0 shape:", R0.shape)              # expect (Kb, Kb)
# evals = np.linalg.eigvalsh((R0 + R0.T) * 0.5)
# print("R0 min eigenvalue:", evals.min())  # should be >= -1e-10 (numeric noise)

# # --- Build two SoF models (no penalty vs. λ>0) ---
# m0 = ScalarOnFunctionModel(basis_pairs, criterion="D", intercept=True, lambda_penalty=None)
# m1 = ScalarOnFunctionModel(basis_pairs, criterion="D", intercept=True, lambda_penalty=1.0)

# runs = 8
# Gamma = np.random.uniform(-1, 1, size=(runs, m0.Kx)).astype(np.float64)

# # Z and M for the base (no-penalty) model
# Z0 = m0.model_matrix(tf.constant(Gamma))
# M0 = m0.information_matrix(Z0).numpy()

# # Check that m1 builds R̃ (p x p) with zero top-left for intercept
# R_tilde = m1.R.numpy()
# print("R̃ shape:", R_tilde.shape, "p=", m1.p)
# print("Intercept penalty entry (should be 0):", R_tilde[0, 0])

# # Verify that regularization equals M + λR̃ (regularization is applied only inside objective/report)
# M_reg_expected = M0 + 1.0 * R_tilde
# M_reg_actual = m1._regularize_information(tf.constant(M0)).numpy()
# print("Regularization close:", np.allclose(M_reg_expected, M_reg_actual, atol=1e-10))

# # Quick criterion sanity (no need for equality, just that λ changes it smoothly)
# loss0 = float(m0.objective(tf.constant(Gamma)).numpy().reshape(()))
# loss1 = float(m1.objective(tf.constant(Gamma)).numpy().reshape(()))
# print("Loss (λ=None):", loss0)
# print("Loss (λ=1.0):", loss1)


In [ ]:
# import numpy as np
# from bases.polynomial import PolynomialBasis
# from inner_products.penalty import PenaltyBuilder

# # Analytic R0 for monomial basis {1, t, ..., t^d}
# def analytic_R_poly(degree: int) -> np.ndarray:
#     K = degree + 1
#     R = np.zeros((K, K), dtype=float)
#     for i in range(2, K):
#         for j in range(2, K):
#             R[i, j] = (i * (i - 1) * j * (j - 1)) / (i + j - 3)
#     return R

# # Build numeric R0
# pB = PolynomialBasis(degree=3)
# pb = PenaltyBuilder([pB], quad_points=256, diff_h=1e-4)
# R0_num = pb.build()

# R0_ref = analytic_R_poly(3)

# print("Numeric R0:\n", np.round(R0_num, 6))
# print("Analytic R0:\n", R0_ref)
# print("Abs diff:\n", np.round(np.abs(R0_num - R0_ref), 8))
# print("Max abs diff:", float(np.max(np.abs(R0_num - R0_ref))))

In [ ]:
# from bases.polynomial import PolynomialBasis
# from models.sof import ScalarOnFunctionModel
# import numpy as np, tensorflow as tf

# xB = PolynomialBasis(3)   # just to get a quick Kx
# bB = PolynomialBasis(3)
# m0 = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True)            # no penalty
# m1 = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True, lambda_penalty=0.1)

# runs = 6
# Gamma = np.random.uniform(-1,1,(runs,m0.Kx)).astype(np.float64)
# print("penalty active?", m1.R is not None, "R̃ shape:", None if m1.R is None else m1.R.shape)
# print("loss (λ=None) vs (λ=0.1):", float(m0.objective(Gamma)), float(m1.objective(Gamma)))


In [ ]:
# # --- Smoke test: Scalar-on-Function + NBDO (no penalty) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from optimizers.nbdo import NBDO

# # Reproducibility
# SEED = 42
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Bases (1 factor for speed; feel free to add more)
# xB = BSplineBasis(degree=3, total_knots_num=12)   # input basis
# bB = BSplineBasis(degree=3, total_knots_num=12)   # parameter basis
# pairs = [(xB, bB)]

# # Model (no penalty)
# model = ScalarOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",       # or "A"
#     intercept=True,
#     lambda_penalty=None  # NO PENALTY
# )

# # NBDO setup
# runs = 30
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)

# # Fit the autoencoder (small epochs for quick smoke)
# opt.fit(epochs=50, batch_size=32, patience=5)

# # BO in latent space (small n_calls for speed)
# report0, design0 = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== NO PENALTY ===")
# print(f"runs = {runs}, Kx = {model.Kx}, Kb = {model.Kb}, p = {model.p}")
# print(f"Optimal report (criterion='A' -> trace(M)): {report0:.6g}")
# print("Design shape:", design0.shape)

# # === NO PENALTY ===
# # runs = 30, Kx = 14, Kb = 14, p = 15
# # Optimal report (criterion='A' -> trace(M)): 4132.03
# # Design shape: (30, 14)

In [ ]:
# # --- Smoke test: Scalar-on-Function + NBDO (no penalty) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from optimizers.nbdo import NBDO

# # Reproducibility
# SEED = 42
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Bases (1 factor for speed; feel free to add more)
# xB = BSplineBasis(degree=3, total_knots_num=12)   # input basis
# bB = BSplineBasis(degree=3, total_knots_num=12)   # parameter basis
# pairs = [(xB, bB)]

# # Model (no penalty)
# model = ScalarOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",       # or "A"
#     intercept=True,
#     lambda_penalty=0.1  # NO PENALTY
# )

# # NBDO setup
# runs = 30
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)

# # Fit the autoencoder (small epochs for quick smoke)
# opt.fit(epochs=50, batch_size=32, patience=5)

# # BO in latent space (small n_calls for speed)
# report0, design0 = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== NO PENALTY ===")
# print(f"runs = {runs}, Kx = {model.Kx}, Kb = {model.Kb}, p = {model.p}")
# print(f"Optimal report (criterion='A' -> trace(M)): {report0:.6g}")
# print("Design shape:", design0.shape)

# # === NO PENALTY ===
# # runs = 30, Kx = 14, Kb = 14, p = 15
# # Optimal report (criterion='A' -> trace(M)): 3.44801
# # Design shape: (30, 14)

In [ ]:
# import numpy as np, tensorflow as tf
# from bases.bspline import BSplineBasis
# from bases.polynomial import PolynomialBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel

# # Predictor bases (η over s)
# xB = BSplineBasis(degree=3, total_knots_num=12)   # c(s)
# bB = PolynomialBasis(degree=3)                    # η(s)  (Kb=4)
# pairs = [(xB, bB)]

# # Response basis θ(t)
# theta = FourierBasis(n_harmonics=3, include_constant=True)  # L = 1 + 2*3 = 7

# m = FunctionOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",
#     intercept=True,
#     response_basis=theta,
#     lambda_s=0.5,            # activate R_s
#     lambda_t=0.25,           # activate S_t & J_HH
# )

# print("Kb =", m.Kb, "p =", m.p, "L =", theta.num_basis())
# print("R_s is None?", m.R_s is None, "shape:", None if m.R_s is None else m.R_s.shape)
# print("S_t is None?", m.S_t is None, "shape:", None if m.S_t is None else m.S_t.shape)
# print("J_HH is None?", m.J_HH is None, "shape:", None if m.J_HH is None else m.J_HH.shape)

# # PSD-ish sanity:
# def min_eig(a): 
#     w = np.linalg.eigvalsh(((a+a.T)/2.0))
#     return float(w.min())

# if m.R_s is not None:
#     R0 = m.R_s.numpy()[1:,1:]  # drop intercept row/col
#     print("min eig R_s (no intercept):", f"{min_eig(R0):.3e}")
# if m.S_t is not None:
#     print("min eig S_t:", f"{min_eig(m.S_t.numpy()):.3e}")
# if m.J_HH is not None:
#     J0 = m.J_HH.numpy()
#     print("J_HH[0,0] (should be 1):", J0[0,0])
#     print("min eig J_HH:", f"{min_eig(J0):.3e}")


In [ ]:
# import numpy as np, tensorflow as tf
# from bases.bspline import BSplineBasis
# from bases.polynomial import PolynomialBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel

# # One predictor
# xB = BSplineBasis(3, 10)       # c(s)
# eta = PolynomialBasis(3)       # η(s), Kb=4
# pairs = [(xB, eta)]
# theta = FourierBasis(2, True)  # L = 1 + 2*2 = 5

# # Same design Γ for all evaluations
# m_base = FunctionOnFunctionModel(pairs, criterion="A", intercept=True, response_basis=theta)
# runs = 8
# Gamma = np.random.uniform(-1,1,(runs, m_base.Kx)).astype(np.float64)

# # No penalty
# loss0 = float(m_base.objective(Gamma))

# # s-penalty only
# m_s = FunctionOnFunctionModel(pairs, criterion="A", intercept=True,
#                               response_basis=theta, lambda_s=0.5, lambda_t=None)
# loss_s = float(m_s.objective(Gamma))

# # t-penalty only
# m_t = FunctionOnFunctionModel(pairs, criterion="A", intercept=True,
#                               response_basis=theta, lambda_s=None, lambda_t=0.5)
# loss_t = float(m_t.objective(Gamma))

# # both penalties
# m_both = FunctionOnFunctionModel(pairs, criterion="A", intercept=True,
#                                  response_basis=theta, lambda_s=0.5, lambda_t=0.5)
# loss_b = float(m_both.objective(Gamma))

# print("A-loss (no pen)   :", loss0)
# print("A-loss (s pen)    :", loss_s, "  <= no pen ? ", loss_s <= loss0)
# print("A-loss (t pen)    :", loss_t, "  <= no pen ? ", loss_t <= loss0)
# print("A-loss (both)     :", loss_b, "  <= each  ? ", loss_b <= min(loss_s, loss_t))


In [ ]:
# import numpy as np, tensorflow as tf

# # Assume you've already built:
# # m_base  = FoF with lambda_s=None, lambda_t=None, response_basis=theta
# # m_t     = FoF with lambda_s=None, lambda_t>0, response_basis=theta
# # Gamma   = some fixed design (runs, m_base.Kx)
# L = m_t.response_basis.num_basis()

# # 1) No-pen A-loss on p×p
# loss0 = float(m_base.objective(Gamma))

# # 2) Lifted baseline A-loss on (L*p)×(L*p): F0 = I_L ⊗ Mz
# Z = m_base.model_matrix(Gamma)
# Mz = m_base.information_matrix(Z)                         # (p,p)
# F0 = tf.linalg.LinearOperatorBlockDiag(
#         [tf.linalg.LinearOperatorFullMatrix(Mz)] * L
#      ).to_dense()                                         # (L*p, L*p)
# loss_F0 = float(m_base.psi.loss_from_M(F0).numpy())

# # 3) t-penalized A-loss (already on (L*p)×(L*p))
# loss_t = float(m_t.objective(Gamma))

# print("A-loss no-pen (p×p):", loss0)
# print("Lifted baseline (I⊗M):", loss_F0, " ~ L*loss0 =", L*loss0)
# print("t-pen A-loss:", loss_t, " <= lifted baseline ? ", loss_t <= loss_F0)


In [ ]:
# m = FunctionOnFunctionModel(pairs, criterion="A", intercept=True,
#                               response_basis=theta, lambda_s=0.1, lambda_t=0.5)
# # Assuming 'm' is a FunctionOnFunctionModel with intercept=True
# if m.R_s is not None:
#     R = m.R_s.numpy()
#     print("R_s[0,0] =", R[0,0], " (expect 0)")
#     print("any intercept row/col nonzero? ", np.any(np.abs(R[0,:])>1e-12) or np.any(np.abs(R[:,0])>1e-12))

# if m.J_HH is not None:
#     JH = m.J_HH.numpy()
#     print("J_HH[0,0] =", JH[0,0], " (expect 1)")
#     print("any off-diag in intercept row/col? ", np.any(np.abs(JH[0,1:])>1e-12) or np.any(np.abs(JH[1:,0])>1e-12))

# if m.S_t is not None and getattr(m, "response_basis", None):
#     # If the response basis includes a constant, its roughness row/col should be ~0
#     # We assume the constant is basis index 0 for Fourier/Polynomial.
#     St = m.S_t.numpy()
#     print("S_t row/col 0 near zero? ",
#           np.all(np.abs(St[0,:])<1e-8) and np.all(np.abs(St[:,0])<1e-8))

In [ ]:
# import numpy as np, tensorflow as tf
# from bases.bspline import BSplineBasis
# from bases.polynomial import PolynomialBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel

# # Two factors with different parameter bases
# x1, eta1 = BSplineBasis(3,12), PolynomialBasis(3)         # Kb1=4
# x2, eta2 = BSplineBasis(3,10), FourierBasis(2, True)      # Kb2=1+2*2=5
# pairs = [(x1, eta1), (x2, eta2)]
# theta = FourierBasis(2, True)  # L=5

# m = FunctionOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",
#     intercept=True,
#     response_basis=theta,
#     lambda_s=0.3,
#     lambda_t=0.2,
# )

# print("Kb total:", m.Kb, " (expect 4+5=9)")
# print("p:", m.p, " (Kb+1 because intercept)")
# print("Shapes → R_s:", None if m.R_s is None else m.R_s.shape,
#       " J_HH:", None if m.J_HH is None else m.J_HH.shape,
#       " S_t:", None if m.S_t is None else m.S_t.shape)

# # Verify block structure (drop intercept row/col)
# R0 = m.R_s.numpy()[1:,1:]
# J0 = m.J_HH.numpy()[1:,1:]
# Kb1, Kb2 = eta1.num_basis(), eta2.num_basis()
# R_off = R0[:Kb1, Kb1:]
# J_off = J0[:Kb1, Kb1:]

# print("‖R_s off-block‖ (≈0):", np.linalg.norm(R_off))
# print("‖J_HH off-block‖ (≈0):", np.linalg.norm(J_off))

# # Dimension of penalized info F_λ: (L*p) x (L*p)
# runs = 6
# Gamma = np.random.uniform(-1,1,(runs, m.Kx)).astype(np.float64)
# Mz = m.information_matrix(m.model_matrix(Gamma))
# Fλ = m._regularize_information(Mz)
# print("F_λ shape:", Fλ.shape, " (expect", theta.num_basis()*m.p, "by same)")

In [ ]:
# from models.fof import FunctionOnFunctionModel
# from bases.bspline import BSplineBasis
# from bases.fourier import FourierBasis
# xB = BSplineBasis(3,12); bB = BSplineBasis(3,12); theta = FourierBasis(2, True)

# # No penalty: identical behavior
# m0 = FunctionOnFunctionModel([(xB,bB)], response_basis=None, lambda_s=None, lambda_t=None)
# print(m0.R_s, m0.S_t, m0.J_HH)  # -> None None None

# # t-penalty without response_basis should fail fast
# try:
#     FunctionOnFunctionModel([(xB,bB)], lambda_t=0.1)
# except Exception as e:
#     print("Expected error:", type(e).__name__, str(e)[:60], "...")


In [ ]:
# Item 11 — Sanity checks (A-loss monotonicity with s- and t- penalties)

import numpy as np, tensorflow as tf

from bases.bspline import BSplineBasis
from bases.polynomial import PolynomialBasis
from bases.fourier import FourierBasis
from models.fof import FunctionOnFunctionModel

# Bases
xB   = BSplineBasis(3, 12)          # c(s)
eta  = PolynomialBasis(3)           # η(s), Kb=4
theta = FourierBasis(2, True)       # θ(t), L = 1 + 2*2 = 5

pairs = [(xB, eta)]

# Fixed design Γ for apples-to-apples comparisons
rng = np.random.default_rng(123)
runs = 10
# Kx = xB.num_basis() here; if multiple factors, use m.Kx after init
Kx = xB.num_basis()
Gamma = rng.uniform(-1, 1, size=(runs, Kx)).astype(np.float64)

# Models: base, s-pen, t-pen, both
m_base = FunctionOnFunctionModel(pairs, criterion="A", intercept=True, response_basis=theta)
m_s    = FunctionOnFunctionModel(pairs, criterion="A", intercept=True, response_basis=theta, lambda_s=0.3)
m_t    = FunctionOnFunctionModel(pairs, criterion="A", intercept=True, response_basis=theta, lambda_t=0.3)
m_both = FunctionOnFunctionModel(pairs, criterion="A", intercept=True, response_basis=theta, lambda_s=0.3, lambda_t=0.3)

# A-loss values on the SAME Γ
loss0 = float(m_base.objective(Gamma))      # tr(Mz^{-1}) on p×p
loss_s = float(m_s.objective(Gamma))        # tr((Mz+λs Rs)^{-1})
loss_t = float(m_t.objective(Gamma))        # tr(Fλ^{-1}) on (L*p)×(L*p)
loss_b = float(m_both.objective(Gamma))     # tr(Fλ^{-1}) with both penalties

# Lifted baseline for t-pen: F0 = I_L ⊗ Mz
Z  = m_base.model_matrix(Gamma)
Mz = m_base.information_matrix(Z)  # (p,p)
L  = theta.num_basis()
F0 = tf.linalg.LinearOperatorBlockDiag(
        [tf.linalg.LinearOperatorFullMatrix(Mz)] * L
     ).to_dense()
loss_F0 = float(m_base.psi.loss_from_M(F0).numpy())   # should be ~ L * loss0

print("A-loss no-pen (p×p):            ", loss0)
print("Lifted baseline (I⊗Mz):         ", loss_F0, "  ~ L*loss0 =", L*loss0)
print("A-loss s-pen <= no-pen?         ", loss_s, " <= ", loss0, " -> ", loss_s <= loss0)
print("A-loss t-pen <= lifted baseline?", loss_t, " <= ", loss_F0, " -> ", loss_t <= loss_F0)
print("A-loss both <= min(s, t scaled)?", loss_b, " <= ", min(loss_s, loss_t), " -> ", loss_b <= min(loss_s, loss_t))

# (Optional) dimension-invariant view: normalize t-pen/both by L and compare to no-pen
print("Scaled t-pen  <= no-pen?        ", (loss_t / L) <= loss0)
print("Scaled both   <= no-pen?        ", (loss_b / L) <= loss0)

In [ ]:
# # Item 11 — Structure: block-diagonal R_s/J_HH, intercept padding, analytic poly R check

# import numpy as np
# from bases.bspline import BSplineBasis
# from bases.polynomial import PolynomialBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel

# # --- A) Multi-factor structure (different Kb per factor) ---
# x1, eta1 = BSplineBasis(3, 12), PolynomialBasis(3)        # Kb1 = 4
# x2, eta2 = BSplineBasis(3, 10), FourierBasis(2, True)     # Kb2 = 1 + 2*2 = 5
# pairs = [(x1, eta1), (x2, eta2)]
# theta = FourierBasis(2, True)  # L=5

# m = FunctionOnFunctionModel(
#     basis_pairs=pairs, criterion="A", intercept=True,
#     response_basis=theta, lambda_s=0.2, lambda_t=0.2
# )

# print("Kb total:", m.Kb, " (expect 4+5=9)")
# print("p:", m.p, " (Kb+1 if intercept)")
# print("Shapes: R_s", None if m.R_s is None else m.R_s.shape,
#       "J_HH", None if m.J_HH is None else m.J_HH.shape,
#       "S_t", None if m.S_t is None else m.S_t.shape)

# # Drop intercept to inspect blocks
# R0 = m.R_s.numpy()[1:, 1:]
# J0 = m.J_HH.numpy()[1:, 1:]
# Kb1, Kb2 = eta1.num_basis(), eta2.num_basis()
# R_off = R0[:Kb1, Kb1:]
# J_off = J0[:Kb1, Kb1:]
# print("‖R_s off-block‖ (≈0):", np.linalg.norm(R_off))
# print("‖J_HH off-block‖ (≈0):", np.linalg.norm(J_off))

# # Intercept padding checks
# print("R_s[0,0] (expect 0):", float(m.R_s.numpy()[0,0]))
# print("J_HH[0,0] (expect 1):", float(m.J_HH.numpy()[0,0]))

# # Response penalty: constant row/col ~ 0 if basis includes constant
# St = m.S_t.numpy()
# print("S_t first row/col near 0?", np.all(np.abs(St[0,:]) < 1e-8) and np.all(np.abs(St[:,0]) < 1e-8))

# # --- B) Analytic polynomial R check for degree=3 ---
# eta_poly = PolynomialBasis(3)  # {1, t, t^2, t^3}
# xB      = BSplineBasis(3, 10)  # arbitrary c(s)
# m_poly  = FunctionOnFunctionModel(
#     basis_pairs=[(xB, eta_poly)], criterion="A", intercept=True,
#     response_basis=theta, lambda_s=0.1, lambda_t=None
# )
# # Extract the (Kb x Kb) block (drop intercept row/col)
# Rpoly = m_poly.R_s.numpy()[1:, 1:]
# # Analytic expectation for degree 3 monomials: bottom-right 2x2 = [[4,6],[6,12]]
# R_expected = np.zeros((4,4))
# R_expected[2,2] = 4.0; R_expected[2,3] = 6.0
# R_expected[3,2] = 6.0; R_expected[3,3] = 12.0

# print("Polynomial R (numeric):\n", np.round(Rpoly, 5))
# print("Polynomial R (analytic):\n", R_expected)
# print("Max |diff|:", float(np.max(np.abs(Rpoly - R_expected))))

In [ ]:
# # --- Smoke test: Function-on-Function + NBDO (NO penalty) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# # Reproducibility
# SEED = 123
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Bases
# xB    = BSplineBasis(degree=3, total_knots_num=12)   # c(s)
# eta   = BSplineBasis(degree=3, total_knots_num=12)   # η(s)
# theta = FourierBasis(n_harmonics=2, include_constant=True)  # θ(t), L = 5

# pairs = [(xB, eta)]

# # Model (no penalties)
# model = FunctionOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",
#     intercept=True,
#     response_basis=theta,   # harmless even if no t-penalty
#     lambda_s=None,
#     lambda_t=None,
# )

# # NBDO
# runs = 10
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)
# opt.fit(epochs=50, batch_size=32, patience=5)
# report0, design0 = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== FoF: NO PENALTY ===")
# print(f"runs={runs}, Kx={model.Kx}, Kb={model.Kb}, p={model.p}, effective_p={model.effective_p}")
# print(f"A-report (trace(M^-1)) = {report0:.6g}")
# print("Design shape:", design0.shape)

# # === FoF: NO PENALTY ===
# # runs=10, Kx=14, Kb=14, p=15, effective_p=15
# # A-report (trace(M^-1)) = 5.00056e+06
# # Design shape: (10, 14)

In [ ]:
# # --- Smoke test: Function-on-Function + NBDO (s-penalty only) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# SEED = 123
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Same bases as Cell 1
# xB    = BSplineBasis(degree=3, total_knots_num=12)   # c(s)
# eta   = BSplineBasis(degree=3, total_knots_num=12)   # η(s)
# theta = FourierBasis(n_harmonics=2, include_constant=True)  # θ(t), L = 5
# pairs = [(xB, eta)]

# # Model (s-penalty active)
# model = FunctionOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",
#     intercept=True,
#     response_basis=theta,     # kept for consistency
#     lambda_s=0.1,             # s-direction roughness
#     lambda_t=None,            # no t-penalty
# )

# # NBDO (same settings)
# runs = 10
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)
# opt.fit(epochs=50, batch_size=32, patience=5)
# report_s, design_s = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== FoF: s-PENALTY (λ_s=0.1) ===")
# print(f"runs={runs}, Kx={model.Kx}, Kb={model.Kb}, p={model.p}, effective_p={model.effective_p}")
# print(f"A-report (trace(M^-1)) = {report_s:.6g}")
# print("Design shape:", design_s.shape)

# # === FoF: s-PENALTY (λ_s=0.1) ===
# # runs=10, Kx=14, Kb=14, p=15, effective_p=75
# # A-report (trace(M^-1)) = 46.7743
# # Design shape: (10, 14)

In [ ]:
# # --- Smoke test: Function-on-Function + NBDO (both s & t penalties) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from bases.fourier import FourierBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# SEED = 123
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Same bases
# xB    = BSplineBasis(degree=3, total_knots_num=12)   # c(s)
# eta   = BSplineBasis(degree=3, total_knots_num=12)   # η(s)
# theta = FourierBasis(n_harmonics=2, include_constant=True)  # θ(t), L = 5
# pairs = [(xB, eta)]

# # Model (both penalties active)
# model = FunctionOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",
#     intercept=True,
#     response_basis=theta,
#     lambda_s=0.1,   # s-direction
#     lambda_t=0.1,   # t-direction
# )

# # NBDO (same settings)
# runs = 10
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)
# opt.fit(epochs=50, batch_size=32, patience=5)
# report_b, design_b = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== FoF: BOTH PENALTIES (λ_s=0.1, λ_t=0.1) ===")
# print(f"runs={runs}, Kx={model.Kx}, Kb={model.Kb}, p={model.p}, effective_p={model.effective_p}")
# print(f"A-report (trace(M^-1)) = {report_b:.6g}")
# print("Design shape:", design_b.shape)

# # === FoF: BOTH PENALTIES (λ_s=0.1, λ_t=0.1) ===
# # runs=10, Kx=14, Kb=14, p=15, effective_p=75
# # A-report (trace(M^-1)) = 10.8715
# # Design shape: (10, 14)

=== FoF: BOTH PENALTIES (λ_s=0.1, λ_t=0.1) ===
runs=10, Kx=14, Kb=14, p=15, effective_p=75
A-report (trace(M^-1)) = 10.8715
Design shape: (10, 14)
